In [3]:
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.objectives import categorical_crossentropy

from keras.models import Model
from keras.utils import generic_utils
from keras.engine import Layer, InputSpec
from keras import initializers, regularizers

In [4]:
class Config:
    
    def __init__(self):
        
        #Print the process or not
        self.verbose = True
        
        #Name of base network
        self.network = 'vgg'
        
        #Settings for data augmentation
        self.use_horizontal_flips = False
        self.use_vertical_flips = False
        self.rot_90 = False
        
        self.anchor_box_scales = [64, 128, 256]
        
        self.anchor_box_ratios = [[1,1], [1./math.sqrt(2), 2/math.sqrt(2)], [2./math.sqrt(2), 1./math.srqt(2)]]
        
        self.im_size = 300
        self.img_channel_mean = [103.939, 116.779, 123.68]
        self.img_scaling_factor = 1.0
        
        self.num_rois = 4
        self.rpn_stride = 16
        
        self.balanced_classes = False
        
        self.std_scaling = 4.0
        
        self.classifier_regr_std = [8.0, 8.0, 4.0, 4.0]
        
        self.rpn_min_overlap = 0.3
        self.rpn_max_overlap = 0.7
        
        self.classifier_min_overlap = 0.1
        self.classifier_max_overlap = 0.5
        
        self.class_mapping = None
        
        self.model_path = None
        
        

In [ ]:
def get_data(input_path):
    
    found_bg = False
    all_imgs = {}
    
    classes_count = {}
    
    class_mapping = {}
    
    visualise = True
    
    i = 1
    
    with open(input_path, 'r') as f:
        
        print('Parsing annotation files')
        
        for line in f:
            
            #Print process
            
            sys.stdout.write('\r' + 'idx=' + str(i))
            i += 1
            
            line_split = line.strip().split(',')
            
            (filename, x1, y1, x2, y2, class_name) = line_split
            
            if class_name not in classes_count:
                classes_count[class_name] = 1
            else:
                classes_count[class_name] += 1
            
            if class_name not in class_mapping:
                if class_name == 'bg' and found_bg == False:
                    print('Found class name with special name bg. Will be treated as a background region (this is usually for hard negative mining).')
                    found_bg = True
                
                class_mapping[class_name] = len(class_mapping)
            
            if filename not in all_imgs:
                all_imgs[filename] = {}
                
                img = cv2.imread(filename)
                (rows, cols) = img.shape[:2]
                all_imgs[filename]['filepath'] = filename
                all_imgs[filename]['width'] = cols
                all_imgs[filename]['height'] = rows
                all_imgs[filename]['bboxes'] = []
                
                
            all_imgs[filename]['bboxes'].append({'class': class_name, 'x1': int(x1), 'x2': int(x2), 'y1': int(y1), 'y2': int(y2)})
            
        all_data = []
        for key in all_imgs:
            all_data.append(all_imgs[key])
            
            if found_bg:
                if class_mapping['bg'] != len(class_mapping) - 1:
                    key_to_switch = [key for key in class_mapping.keys() if class_mapping[key] == len(class_mapping) -1][0]